<a href="https://www.kaggle.com/code/dilekdd/association-rule-based-recommender-de?scriptVersionId=197933347" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<div style="text-align: center; font-size: 40px; font-weight: bold; color: hotpink;">
     Association Rule Based Recommender Germany
</div>

Below are the basket information of 3 different users. Make the most appropriate product suggestion for this basket information using the association rule. 
Product suggestions can be 1 or more than 1. Derive the decision rules from 2010-2011 Germany customers.

The ID of the product in User 1's basket: 21987 
The ID of the product in User 2's basket: 23235 
The ID of the product in User 3's basket: 22747



The dataset named Online Retail II includes online sales transactions of a UK-based retail company between 01/12/2009 - 09/12/2011. The company's product catalogue includes gift items and it is known that most of its customers are wholesalers.

| **Column**     | **Description**                                                                    |
|----------------|------------------------------------------------------------------------------------|
| InvoiceNo      | Invoice number (If the code starts with 'C', it indicates the transaction was canceled). |
| StockCode      | Product code (Unique for each product).                                             |
| Description    | Product name.                                                                      |
| Quantity       | Product quantity (How many of each product were sold in the invoice).               |
| InvoiceDate    | Invoice date.                                                                      |
| UnitPrice      | Invoice price (in British pounds).                                                  |
| CustomerID     | Unique customer number.                                                            |
| Country        | Country name.                                                                      |


In [1]:
!pip install mlxtend
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
pd.set_option('display.width', 500)
# çıktının tek bir satırda olmasını sağlar.
pd.set_option('display.expand_frame_repr', False)
from mlxtend.frequent_patterns import apriori, association_rules

In [2]:
df_ = pd.read_excel("/kaggle/input/online-retail-ii/online_retail_II.xlsx",
                    sheet_name="Year 2010-2011", engine="openpyxl")

In [3]:
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      541910 non-null  object        
 1   StockCode    541910 non-null  object        
 2   Description  540456 non-null  object        
 3   Quantity     541910 non-null  int64         
 4   InvoiceDate  541910 non-null  datetime64[ns]
 5   Price        541910 non-null  float64       
 6   Customer ID  406830 non-null  float64       
 7   Country      541910 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [5]:
df.describe(include=[np.number]).T

,count,mean,std,min,25%,50%,75%,max
Quantity,541910.0,9.552234,218.080957,-80995.00,1.00,3.00,10.00,80995.0
Price,541910.0,4.611138,96.759765,-11062.06,1.25,2.08,4.13,38970.0
Customer ID,406830.0,15287.684160,1713.603074,12346.00,13953.00,15152.00,16791.00,18287.0


In [6]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [7]:
df.shape

(541910, 8)

In [8]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

def retail_data_prep(dataframe):
    dataframe.dropna(inplace=True)
    dataframe["Invoice"] = dataframe["Invoice"].astype(str)
    dataframe = dataframe[~dataframe["Invoice"].str.contains("C", na=False)]
    dataframe = dataframe[dataframe["Quantity"] > 0]
    dataframe = dataframe[dataframe["Price"] > 0]
    replace_with_thresholds(dataframe, "Quantity")
    replace_with_thresholds(dataframe, "Price")
    return dataframe

df = retail_data_prep(df)

/tmp/ipykernel_17/2447131211.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-177.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit


In [9]:
df.describe(include=[np.number]).T

,count,mean,std,min,25%,50%,75%,max
Quantity,397885.0,11.830770,25.523052,1.000,2.00,6.00,12.00,298.50
Price,397885.0,2.893492,3.227175,0.001,1.25,1.95,3.75,37.06
Customer ID,397885.0,15294.416882,1713.144421,12346.000,13969.00,15159.00,16795.00,18287.00


In [10]:
df_de = df[df['Country'] == "Germany"]

In [11]:
df_de.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6.0,2010-12-01 13:04:00,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6.0,2010-12-01 13:04:00,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12.0,2010-12-01 13:04:00,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12.0,2010-12-01 13:04:00,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12.0,2010-12-01 13:04:00,1.95,12662.0,Germany


In [12]:
df_de.shape

(9040, 8)

In [13]:
def create_invoice_product_df(dataframe, id=False):
    if id:
        return dataframe.groupby(['Invoice', "StockCode"])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
    else:
        return dataframe.groupby(['Invoice', 'Description'])['Quantity'].sum().unstack().fillna(0). \
            applymap(lambda x: 1 if x > 0 else 0)
    
de_inv_pro_df = create_invoice_product_df(df_de, id=True)

/tmp/ipykernel_17/4152941681.py:4: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  applymap(lambda x: 1 if x > 0 else 0)


In [14]:
def check_id(dataframe, stock_code):
    product_name = dataframe[dataframe["StockCode"] == stock_code][["Description"]].values[0].tolist()
    print(product_name)
    
check_id(df_de, 21987)
check_id(df_de, 23235)
check_id(df_de, 22747)

['PACK OF 6 SKULL PAPER CUPS']
['STORAGE TIN VINTAGE LEAF']
["POPPY'S PLAYHOUSE BATHROOM"]


In [15]:
frequent_itemsets = apriori(de_inv_pro_df,
                            min_support=0.01,
                            use_colnames=True)

frequent_itemsets.sort_values("support", ascending=False)

rules = association_rules(frequent_itemsets,
                          metric="support",
                          min_threshold=0.01)

rules[(rules["support"]>0.05) & (rules["confidence"]>0.1) & (rules["lift"]>5)]. \
sort_values("lift", ascending=False)

/opt/conda/lib/python3.10/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
6154,"(20719, POST)",(20724),0.115974,0.070022,0.054705,0.471698,6.736439,0.046584,1.760316,0.963267
6155,(20724),"(20719, POST)",0.070022,0.115974,0.054705,0.781250,6.736439,0.046584,4.041263,0.915671
264,(20724),(20719),0.070022,0.126915,0.059081,0.843750,6.648168,0.050194,5.587746,0.913551
265,(20719),(20724),0.126915,0.070022,0.059081,0.465517,6.648168,0.050194,1.739959,0.973081
6157,(20719),"(20724, POST)",0.126915,0.065646,0.054705,0.431034,6.566092,0.046373,1.642199,0.970927
6152,"(20724, POST)",(20719),0.065646,0.126915,0.054705,0.833333,6.566092,0.046373,5.238512,0.907260


In [16]:
def arl_recommender(rules_df, product_id, rec_count=1):
    sorted_rules = rules_df.sort_values("lift", ascending=False)
    recommendation_list = []
    for i, product in enumerate(sorted_rules["antecedents"]):
        for j in list(product):
            if j == product_id:
                recommendation_list.append(list(sorted_rules.iloc[i]["consequents"])[0])

    return recommendation_list[0:rec_count]

In [17]:
product_ids = [21987, 23235, 22747]
rec_counts = [1, 2, 3]

for p_id, r_count in zip(product_ids, rec_counts):
    recommendations = arl_recommender(rules, p_id, r_count)
    print(f"Recommendations for the product {p_id}: {recommendations}")

Recommendations for the product 21987: [21086]
Recommendations for the product 23235: [23244, 23243]
Recommendations for the product 22747: [22745, 22745, 22746]
